# Faster RCNN Inception Resnet V2 training

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
! cp PATH/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download -d omkarnadkarni/lisa-traffic-sign

100% 7.96G/7.99G [00:53<00:00, 239MB/s]
100% 7.99G/7.99G [00:53<00:00, 160MB/s]


In [4]:
! unzip lisa-traffic-sign.zip

Streaming output truncated to the last 5000 lines.
  inflating: negatives/negativePics/nosign09515.png  
  inflating: negatives/negativePics/nosign09516.png  
  inflating: negatives/negativePics/nosign09517.png  
  inflating: negatives/negativePics/nosign09518.png  
  inflating: negatives/negativePics/nosign09519.png  
  inflating: negatives/negativePics/nosign09520.png  
  inflating: negatives/negativePics/nosign09521.png  
  inflating: negatives/negativePics/nosign09522.png  
  inflating: negatives/negativePics/nosign09523.png  
  inflating: negatives/negativePics/nosign09524.png  
  inflating: negatives/negativePics/nosign09525.png  
  inflating: negatives/negativePics/nosign09526.png  
  inflating: negatives/negativePics/nosign09527.png  
  inflating: negatives/negativePics/nosign09528.png  
  inflating: negatives/negativePics/nosign09529.png  
  inflating: negatives/negativePics/nosign09530.png  
  inflating: negatives/negativePics/nosign09531.png  
  inflating: negatives/negative

## Setting up package versions
Faster RCNN Inception Resnet V2 isn't available for Tensorflow 2 so you should downgrade to some version of tensorflow 1. I went with 1.15.0

In [ ]:
! pip install tensorflow-gpu==1.15.0
! pip install numpy==1.21.6

## Installing Object detection modules
I would personally recommend installing models to your google drive. This is just more convenient

In [ ]:
# you need to make branch is compatible with your version of tensorflow
! git clone -b r1.13.0 https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 72856, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 72856 (delta 14), reused 14 (delta 5), pack-reused 72825
Receiving objects: 100% (72856/72856), 579.24 MiB | 30.91 MiB/s, done.
Resolving deltas: 100% (51585/51585), done.


In [ ]:
! cd models/research
! python setup.py build
! python setup.py install
! cd  slim
! python setup.py build
! python setup.py install

Install the pretrained model. We will be using this to initialize our weights

In [ ]:
! wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28.tar.gz
! tar -xvzf faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28.tar.gz

## Generate tf records

In [ ]:
from PIL import Image
import tensorflow as tf
from object_detection.utils import dataset_util
import io
def get_data(name, df,ids):
    with tf.gfile.GFile(name, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = name.encode('utf8')
    image_format = b'png'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    

    for index, row in df.iterrows():
        xmins.append(row["Upper left corner X"] / width)
        xmaxs.append(row["Lower right corner X"] / width)
        ymins.append(row["Upper left corner Y"] / height)
        ymaxs.append(row["Lower right corner Y"] / height)
        tmpc = ids[ids["Name"] == row["Annotation tag"]]["ID"].tolist()[0]
        classes_text.append(row["Annotation tag"].encode('utf8'))
        classes.append(tmpc+1)
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv("allAnnotations.csv")
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(df, test_size=0.25)

In [ ]:
output_path = "lib/datasets/train.record"
groups = train.groupby("Filename")
writer = tf.python_io.TFRecordWriter(output_path)
ids = pd.read_csv("lib/datasets/ids.csv")
for name, group in groups:
    tf_example = get_data(name, group, ids)
    writer.write(tf_example.SerializeToString())
writer.close()

In [ ]:
output_path = "lib/datasets/val.record"
groups = val.groupby("Filename")
writer = tf.python_io.TFRecordWriter(output_path)
ids = pd.read_csv("lib/datasets/ids.csv")
for name, group in groups:
    tf_example = get_data(name, group, ids)
    writer.write(tf_example.SerializeToString())
writer.close()

## Set up config file
The config file I used can be found in lib/faster_rcnn_inception_resnet_v2_atrous_coco.config

You might want to adjust some of the paths depending on where you choose to store your files. These would be fine_tune_checkpoint under train_config and input_path and label_map_path under train_input_reader and eval_input_reader

In [ ]:
class Config:
    def __init__(self):
        self.pipeline_config_path = "lib/faster_rcnn_inception_resnet_v2_atrous_coco.config"
        self.num_train_steps = None
        self.eval_on_train_data = False
        self.sample_1_of_n_eval_examples = 1
        self.sample_1_of_n_eval_on_train_examples = 5
        self.model_dir = "lib/models/model"
        self.checkpoint_dir = None
        self.eval_timeout = 3600
        self.use_tpu = False
        self.tpu_name = None
        self.num_workers = 1
        self.checkpoint_every_n = 1000
        self.record_summaries = True
        self.run_once = False
        self.max_eval_retries = 0
        self.task = 0
        self.master = ""
        self.num_clones = 1
        self.clone_on_cpu = False
        self.worker_replicas =1
        self.ps_tasks = 0
        self.train_dir = "lib/models/train"
        self.train_config_path = ""
        self.input_config_path = ""
        self.model_config_path = ""

## Train the model

In [ ]:
import functools
import json
import os
import tensorflow as tf

from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util
def main(C):
    if C.task == 0:
        tf.gfile.MakeDirs(C.train_dir)
    if C.pipeline_config_path:
        configs = config_util.get_configs_from_pipeline_file(C.pipeline_config_path)
        if C.task == 0:
            tf.gfile.Copy(C.pipeline_config_path,os.path.join(C.train_dir, 'pipeline.config'),overwrite=True)
    else:
        configs = config_util.get_configs_from_multiple_files(
            model_config_path=C.model_config_path,
            train_config_path=C.train_config_path,
            train_input_config_path=C.input_config_path)
        if C.task == 0:
            for name, config in [('model.config', C.model_config_path),
                                ('train.config', C.train_config_path),
                                ('input.config', C.input_config_path)]:
                tf.gfile.Copy(config, os.path.join(C.train_dir, name),overwrite=True)
    model_config = configs['model']
    train_config = configs['train_config']
    input_config = configs['train_input_config']
    model_fn = functools.partial(
        model_builder.build,
        model_config=model_config,
        is_training=True)
    def get_next(config):
        return dataset_builder.make_initializable_iterator(dataset_builder.build(config)).get_next()
    create_input_dict_fn = functools.partial(get_next, input_config)

    env = json.loads(os.environ.get('TF_CONFIG', '{}'))
    cluster_data = env.get('cluster', None)
    cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
    task_data = env.get('task', None) or {'type': 'master', 'index': 0}
    task_info = type('TaskSpec', (object,), task_data)
    # Parameters for a single worker.
    ps_tasks = 0
    worker_replicas = 1
    worker_job_name = 'lonely_worker'
    task = 0
    is_chief = True
    master = ''
    if cluster_data and 'worker' in cluster_data:
        # Number of total worker replicas include "worker"s and the "master".
        worker_replicas = len(cluster_data['worker']) + 1
    if cluster_data and 'ps' in cluster_data:
        ps_tasks = len(cluster_data['ps'])

    if worker_replicas > 1 and ps_tasks < 1:
        raise ValueError('At least 1 ps task is needed for distributed training.')

    if worker_replicas >= 1 and ps_tasks > 0:
        # Set up distributed training.
        server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                             job_name=task_info.type,
                             task_index=task_info.index)
        if task_info.type == 'ps':
            server.join()
            return

        worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
        task = task_info.index
        is_chief = (task_info.type == 'master')
        master = server.target

    graph_rewriter_fn = None
    if 'graph_rewriter_config' in configs:
        graph_rewriter_fn = graph_rewriter_builder.build(
            configs['graph_rewriter_config'], is_training=True)

    trainer.train(
        create_input_dict_fn,
        model_fn,
        train_config,
        master,
        task,
        C.num_clones,
        worker_replicas,
        C.clone_on_cpu,
        ps_tasks,
        worker_job_name,
        is_chief,
        C.train_dir,
        graph_hook_fn=graph_rewriter_fn)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [ ]:
C = Config()
main(C)

## Create inference Graph
You want to make sure the XXXX in lib/models/train/model.ckpt-XXXX is the highest number checkpoint in lib/models/train

In [ ]:
! python models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path lib/faster_rcnn_inception_resnet_v2_atrous_coco.config --trained_checkpoint_prefix lib/models/train/model.ckpt-XXXX --output_directory lib/models/inference_graph